## The command to copy data from blob to compute

az storage blob directory download -c am100-data --destination-path "." --source-path "proto_1" --account-name aq1ds2storage --recursive

In [ ]:
# %pip install opencv-python albumentations pycocotools

In [ ]:
import os
import torch
import torchvision
import numpy as np
import pandas as pd
import cv2
import albumentations as A
import matplotlib.pyplot as plt
from albumentations.pytorch.transforms import ToTensorV2
from utils.engine import train_one_epoch, evaluate

## Configure Path

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

DATABASE_BASE = "../database/proto_1/"
train_csv_path = DATABASE_BASE + "train.csv"
test_csv_path = DATABASE_BASE + "val.csv"
train_image_path = DATABASE_BASE + "train"
test_image_path = DATABASE_BASE + "val"
model_save_path = "trained_model/gray_proto_1/"
model_save_name = model_save_path + "rcnn_b2lr001m8"
pretrained_model = "trained_model/gray_proto_1/fasterrcnn_resnet50_fpn_4ep_best.pt"

## Check data

In [ ]:
train_csv = pd.read_csv(train_csv_path)
print(train_csv.shape)
train_csv.head()

In [ ]:
filename_un = train_csv["filename"].unique()
print(len(filename_un))

In [ ]:
test_csv = pd.read_csv(test_csv_path)
print(test_csv.shape)
test_csv.head()

In [ ]:
categories = train_csv["class"].unique()
print(categories)

## Encoding classes to integers
* 0 is for background

In [ ]:
# Encoding functions
class LabelMap:
    def __init__(self, categories):
        self.map_dict = {}
        self.reverse_map_dict={}
        for i, cat in enumerate(categories):
            self.map_dict[cat] = i + 1
            self.reverse_map_dict[i] = cat
    def fit(self, df, column):
        df[column] = df[column].map(self.map_dict)
        return df
    def inverse(self, df, column):
        df[column] = df[column].map(self.map_dict)
        return df

In [ ]:
label_map = LabelMap(categories)

In [ ]:
train_csv = label_map.fit(train_csv, "class")
train_csv.head()

In [ ]:
test_csv = label_map.fit(test_csv, "class")
test_csv.head()

## Torch Dataset Creation

In [ ]:
class AnimalDataset(torch.utils.data.Dataset):
    def __init__(self, df, image_path, categories, transforms=None,**kwargs):
        super().__init__(**kwargs)
        self.df = df
        self.image_path = image_path
        self.categories = categories
        self.images = self.df["filename"].unique()
        self.transforms = transforms
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image_file = os.path.join(self.image_path, self.images[idx])
        img = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.astype(np.float32)
        img = img/255.0
        image_data = self.df[self.df['filename'] == self.images[idx]]
        labels = torch.as_tensor(image_data["class"].values, dtype=torch.int64)
        xmins = image_data["xmin"].values
        ymins = image_data["ymin"].values
        xmaxs = image_data["xmax"].values
        ymaxs = image_data["ymax"].values
        boxes = torch.as_tensor(np.stack([xmins, ymins, xmaxs, ymaxs], axis=1), dtype=torch.float32)
        areas = (boxes[:,3] - boxes[:,1]) * (boxes[:,2] - boxes[:,0])
        areas = torch.as_tensor(areas, dtype=torch.float32)
        image_id = torch.tensor([idx])
        iscrowd = torch.zeros((len(labels),), dtype=torch.int64)
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = areas
        target["iscrowd"] = iscrowd
        if self.transforms is not None:
            transformed = self.transforms(image=img, bboxes=boxes, labels=labels)
            img = transformed["image"]
            target["boxes"] = torch.as_tensor(transformed["bboxes"],dtype=torch.float32)
        return torch.as_tensor(img, dtype=torch.float32), target
    def get_height_and_width(self, image):
        image_data = self.df.loc[self.df['filename'] == image]
        return image_data["width"].values[0], image_data["height"].values[0]

## Data Augmentation

In [ ]:
transform_train = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.augmentations.geometric.transforms.Affine(scale=0.5, p=0.3,shear=(20), translate_percent=0.2),
    # A.ISONoise(p=0.2),
    # A.GaussNoise(p=0.1),
    # A.CLAHE(p=0.1),
    # A.CenterCrop(height=700, width=1000, p=0.3),
    # A.HueSaturationValue(p=0.1),
    ToTensorV2(p=1)
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))

In [ ]:
transform_test = A.Compose([
    ToTensorV2(p=1)
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))

## Dataloader creation

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [ ]:
train_dataset = AnimalDataset(train_csv, train_image_path, categories, transform_train)
test_dataset = AnimalDataset(test_csv, test_image_path, categories, transform_test)

In [ ]:
data_loader_train = torch.utils.data.DataLoader(
        train_dataset, batch_size=2, shuffle=True, num_workers=4,
        collate_fn=collate_fn)
    
data_loader_test = torch.utils.data.DataLoader(
    test_dataset, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=collate_fn)

## Plot images from dataloader

In [ ]:
def plot_images(images, targets):
    for image, target in zip(images, targets):
        sample = image.permute(1,2,0).cpu().numpy()
        fig, ax = plt.subplots(1, 1, figsize=(8, 4))
        boxes = target["boxes"].cpu().numpy().astype(np.int32)
        for box in boxes:
            cv2.rectangle(sample,
                      (box[0], box[1]),
                      (box[2], box[3]),
                      (220, 0, 0), 3)
        ax.set_axis_off()
        ax.imshow(sample)

In [ ]:
images, targets = next(iter(data_loader_train))
print("Train batch")
plot_images(images, targets)

In [ ]:
images, targets = next(iter(data_loader_test))
print("test batch")
plot_images(images, targets)

## Training

import model from torchvision library </br>
https://pytorch.org/vision/0.11/models.html

In [ ]:
num_classes = len(categories)+1 # add background class

In [ ]:
import torchvision.models.detection as torch_det

detection_model = torch_det.fasterrcnn_resnet50_fpn(pretrained=True)

# ... Alternatives ...
# detection_model = torch_det.fasterrcnn_mobilenet_v3_large_320_fpn(pretrained=True)
# detection_model = torch_det.fasterrcnn_mobilenet_v3_large_fpn(pretrained=True)
# detection_model = torch_det.ssdlite320_mobilenet_v3_large(num_classes=num_classes)
# detection_model = torch_det.ssd300_vgg16(num_classes=num_classes)
# detection_model = torch_det.retinanet_resnet50_fpn(num_classes=num_classes)

adjust classes

In [ ]:
# for FastRCNN series... 
in_features = detection_model.roi_heads.box_predictor.cls_score.in_features
detection_model.roi_heads.box_predictor = torch_det.faster_rcnn.FastRCNNPredictor(in_features, num_classes)

GPU load try and mount the model

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
# if you have pre-trianed model
detection_model.load_state_dict(torch.load(pretrained_model))

detection_model.to(device)

Training and validation config

In [ ]:
import wandb
# from utils.wandbutils import log_to_wandb 
hm_api_key = "5bc9ab59d9422a7fc33e19d0de9014cda1ce4233"
wandb.login(key=hm_api_key)

# Initialize a new W&B run
wandb.init(
    project="AM100_detection_RCNN", 
    name='proto_1_gray',
)

# log config file to W&B
# wandb.save("data/hyp.scratch.custom.yaml")

# Add other hyperparameters as needed
wandb.config.epochs = 100
wandb.config.batch_size = 2
wandb.config.input_image_size = 1300
wandb.config.learning_rate = 0.01
wandb.config.momentum = 0.8
wandb.config.weight_decay = 0.0005

wandb.watch(detection_model, log_freq=100)

In [ ]:
def log_to_wandb(train_result, eval_result):
    lr_f = float(str(train_result.meters['lr']).split()[0])
    loss_f = float(str(train_result.meters['loss']).split()[0])
    loss_classifier_f = float(str(train_result.meters['loss_classifier']).split()[0])
    loss_box_reg_f = float(str(train_result.meters['loss_box_reg']).split()[0])
    loss_objectness_f = float(str(train_result.meters['loss_objectness']).split()[0])
    loss_rpn_box_reg_f = float(str(train_result.meters['loss_rpn_box_reg']).split()[0])

    ap75 = eval_result["(AP)0.75"]
    ap50 = eval_result["(AP)0.50"]
    ap5095 = eval_result["(AP)0.50:0.95"]
    ar5095 = eval_result["(AR)0.50:0.95"]

    wandb.log({"learning rate":lr_f,
               "loss":loss_f,
               "loss_classifier":loss_classifier_f,
               "loss_box_reg":loss_box_reg_f,
               "loss_objectness":loss_objectness_f,
               "loss_rpn_box_reg":loss_rpn_box_reg_f,
               "Avg Precision 0.75":ap75,
               "Avg Precision 0.50":ap50,
               "Avg Precision 0.50:0.95":ap5095,
               "Avg Recall 0.50:0.95":ar5095,})
    
    if lr_f == 0 :
        return True
    return False

In [ ]:
def training(model, train_loader, val_loader, epochs=10, patience=20):
    # init valiables
    cur_patience = patience
    min_loss = float('inf')
    
    # construct an optimizer
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.01,
                                momentum=0.8, weight_decay=0.0005)
    # and a learning rate scheduler
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                   step_size=3,
                                                   gamma=0.1)
    for epoch in range(epochs):
        # train for one epoch, printing every 10 iterations
        train_result = train_one_epoch(model, optimizer, train_loader, device, epoch, print_freq=120)
        # update the learning rate
        lr_scheduler.step()
        # evaluate on the test dataset
        eval_result = evaluate(model, val_loader, device=device)

        # wandb log
        lr_is_zero = log_to_wandb(train_result, eval_result.eval_result)

        # early stopping function
        cur_loss = float(str(train_result.meters['loss']).split()[0])

        if cur_loss < min_loss : 
            min_loss = cur_loss # update min loss
            torch.save(detection_model.state_dict(), model_save_name+'_best.pt') # save the best model
            cur_patience = patience # reset patience
            print("reset patience, saved model")
        else :
            cur_patience -= 1
            print("current patience:",cur_patience, "current loss: ",cur_loss, "best loss: ", min_loss)

        if cur_patience == 0 or lr_is_zero : # reach early stop
            print("early stop at epoch {} due to no loss improve for {} epochs".format(epoch,patience))
            wandb.config.epochs = epoch
            break

In [ ]:
# Create directory if not exist
os.makedirs(model_save_path, exist_ok=True)

# start training
training(detection_model, data_loader_train, data_loader_test, epochs=wandb.config.epochs)

## SAVE MODEL

In [ ]:
# Save model (parameters only)
# torch.save(detection_model.state_dict(), model_save_name+'_last.pt')